# 17 • Consulta de tweets

En este notebook se revisará cómo consultar tweets usando Python.

## Contenido
1. Intro  
   1.1. Twitter API   
   1.2. Librería `GetOldTweets3`  
   1.3. Librería `snscrape`  
2. Referencias  

In [2]:
# Import basic libraries
import pandas as pd
import numpy as np
import altair as alt

## 1. Intro
Existen distintas librerías para consultar tweets y cada una de ellas tiene sus ventajas y desventajas. La manera oficial es hacerlo a través de la aplicación oficial de Twitter obteniendo acceso como desarrollador (`Twitter API`) para la cual se requiere solicitar una cuenta. Existen algunas alternativas que en lo personal he utilizado, aunque para usarlas hay que familiarizarse con ellas, como `GetOldTweets3` y `snscrape` pues no están tan bien documentadas.

Es importante mencionar que las consultas de twitter usualmente arrojan información en formato `json`, los cuales siguen una estructura parecida a los diccionarios de Python y, en caso de guardar estos archivos, se recomienda guardarlos en formato `.json` en el lugar de convertirlos a `.xls`, `.csv`, `.txt` o `npy` para no perder información.

### 1.1 Twitter API
Esta cuenta no tiene costo, aunque sí existe limitación en número de tweets consultados por día. El primer paso es [solicitar una cuenta de desarrollador](https://developer.twitter.com/en/portal/dashboard). Para quien esté interesado en obtener una cuenta les comparto un par de links que les podrían ser útiles

- [Twitter API Data Collection](https://www.youtube.com/watch?v=Jl-_dDqSaUQ&t=59s) por Stevesie Data, ver del minuto 0:59 al 1:44.
- [Developer account support](https://developer.twitter.com/en/support/twitter-api/developer-account) por Twitter.

Una vez que hayan obtenido su solicitud para abrir cuenta de desarrollador haya sido aceptada, podrán encontrar su llave de autenticación en este [link](https://developer.twitter.com/en/portal/projects-and-apps): `API key`, `API secret key`, `Access token` y `Access token secret`, y con ellas podrán tener acceso con distintos programas, entre ellos con Python.

La API de Twitter algunas limitaciones para consultar tweets descritas en este [link](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets).

⚠️ __Desventajas__, el obtener una cuenta de desarrollador en Twitter puede tardar tiempo y además de tener restricciones como un tiempo máximo de días previos de consulta (e.g. no se puede consultar tweets de principios de año), además de un número de existir un máximo número de tweets.

### 1.2 Librería `GetOldTweets3`
Esta es una librería está relativamente bien documentada donde se pueden conusltar tweets relativamente antiguos. Para más información pueden consultar la [descripción del paquete](https://pypi.org/project/GetOldTweets3/) con las indicaciones para su instalación y la [página del paquete en GitHub](https://github.com/Mottl/GetOldTweets3).

Es importante mencionar que actualmente este paquete no funciona.

In [4]:
## librerías
# !pip install GetOldTweets3 #<- installa la librería
import GetOldTweets3 as got

  Using cached GetOldTweets3-0.0.11-py3-none-any.whl (13 kB)
  Using cached pyquery-1.4.3-py3-none-any.whl (22 kB)
  Using cached cssselect-1.2.0-py2.py3-none-any.whl (18 kB)


In [5]:
# Partámetros para consulta
tweetCriteria = got.manager.TweetCriteria().setUsername("JustinTrudeau")\
                                           .setTopTweets(True)\
                                           .setMaxTweets(10)

In [6]:
# esto se almacena en un objeto
tweetCriteria

In [8]:
# # Al realizar la consulta arroja error
# tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
# print(tweet.text)

⚠️ __Desventajas__, actualmente este paquete no funciona de forma correcta. En la siguiente liga se comparte el error, el cual ha sido persistente desde septiembre 2020 https://github.com/Mottl/GetOldTweets3/issues/98

### 1.3 Librería `snscrape`
Esta librería es la mejor alternativa que he utilizado para bajar tweets, aunque aún falta mejorar la documentación. Además, esta librería tiene módulos específicos que sirven para consultar información de otras redes sociales entre las que se encuentran Facebook, Instagram y Telegram.

#### Ejemplo: Obtener tweets relacionados con el **COVID** realizados desde las cuentas de los dirigentes de México, USA y Canadá

In [11]:
# librerías
import snscrape.modules.twitter as sntwitter

In [12]:
# parametros
maxTweets = 2_000
date_initial = "2020-01-01"
date_final = "2022-10-31"

# twitter's accounts
cuentas=['JustinTrudeau', 'JoeBiden', 'lopezobrador_']

In [13]:
%%time
# Parámetros

# Creating list to append tweet data to
tweets_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
print("\n--- START: TwitterSearchScraper por cuenta de Twitter ---\n")
for a in cuentas:
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('"covid" since:'+date_initial+' until:'+date_final+' from:'+a).get_items()):
        if i>maxTweets-1:
            break
        tweets_list.append([tweet.user.username, tweet.date, tweet.id, tweet.content, tweet.url, tweet.lang,
                    tweet.hashtags, tweet.likeCount, tweet.replyCount, tweet.retweetCount, tweet.quoteCount])
        # Otra info que puede sernos útil: tweet.media,  tweet.url
    print("Se identificaron {0} tweets de la cuenta {1} con los parametros dados.".format(i, a))
print("\n--- END: TwitterSearchScraper ---\n\nTIMING")


--- START: TwitterSearchScraper por cuenta de Twitter ---

Se identificaron 1231 tweets de la cuenta JustinTrudeau con los parametros dados.
Se identificaron 255 tweets de la cuenta JoeBiden con los parametros dados.
Se identificaron 2 tweets de la cuenta lopezobrador_ con los parametros dados.

--- END: TwitterSearchScraper ---

TIMING
CPU times: user 885 ms, sys: 114 ms, total: 999 ms
Wall time: 35.8 s


In [14]:
# Pandas dataframe con tweets de los tres presidentes relacionados con el Covid-19
column_names = ("username","date","id","content","url","language","hashtags",
                "likes_count","reply_count","retweet_count","quote_count")
df = pd.DataFrame(tweets_list, columns=column_names)
df

,username,date,id,content,url,language,hashtags,likes_count,reply_count,retweet_count,quote_count
0,JustinTrudeau,2022-10-21 18:16:58+00:00,1583522748642435072,And to Premier @JJHorgan: Thank you for your s...,https://twitter.com/JustinTrudeau/status/15835...,en,None,303,76,53,3
1,JustinTrudeau,2022-10-21 18:16:44+00:00,1583522691675729920,Et au PM @JJHorgan : merci pour les services q...,https://twitter.com/JustinTrudeau/status/15835...,fr,None,109,18,24,0
2,JustinTrudeau,2022-09-26 22:19:08+00:00,1574523994568818688,"Update: Effective October 1st, we’re removing ...",https://twitter.com/JustinTrudeau/status/15745...,en,None,4586,3491,683,413
3,JustinTrudeau,2022-09-26 22:19:02+00:00,1574523971244265472,"Nouveau : à partir du 1er octobre, on lève les...",https://twitter.com/JustinTrudeau/status/15745...,fr,None,181,96,34,10
4,JustinTrudeau,2022-09-22 20:16:22+00:00,1573043547984609284,It’s time – if you’re eligible for your booste...,https://twitter.com/JustinTrudeau/status/15730...,en,None,3648,7427,591,776
...,...,...,...,...,...,...,...,...,...,...,...
1486,JoeBiden,2020-03-15 14:46:19+00:00,1239201266410164224,If you're exhibiting symptoms of COVID-19 — or...,https://twitter.com/JoeBiden/status/1239201266...,en,None,5623,372,1371,123
1487,JoeBiden,2020-03-15 14:43:52+00:00,1239200647616004096,The right to vote is the most sacred American ...,https://twitter.com/JoeBiden/status/1239200647...,en,None,17620,3667,3923,2648
1488,lopezobrador_,2021-01-25 00:30:45+00:00,1353500519344578560,Lamento informarles que estoy contagiado de CO...,https://twitter.com/lopezobrador_/status/13535...,es,None,115615,55106,28781,24896
1489,lopezobrador_,2020-12-27 02:24:20+00:00,1343019855272931336,Nos reunimos con la jefa de Gobierno e integra...,https://twitter.com/lopezobrador_/status/13430...,es,None,13179,1774,3189,323


In [17]:
# ejemplo de tweet
df['content'][0]

'And to Premier @JJHorgan: Thank you for your service to British Columbians and to all Canadians – particularly your leadership on reconciliation with Indigenous Peoples, and helping people during the COVID-19 pandemic. I’m wishing you the very best. https://t.co/X81iiqnqaW'

In [18]:
# Número de comentarios por presidente/primer ministro
print("\nTweets de presidente o primer ministro relacionados con el Covid-19, de enero 2020 a octubre 2022:\n")
pd.DataFrame(df['username'].value_counts()).reset_index().rename(columns={'index':'President', 'username':"tweets"})


Tweets de presidente o primer ministro relacionados con el Covid-19, de enero 2020 a octubre 2022:



,President,tweets
0,JustinTrudeau,1232
1,JoeBiden,256
2,lopezobrador_,3


#### Ejemplo: Obtener tweets con hashtag #INENoSeToca

In [19]:
%%time

# Parámetros
tweets_list_ine = []
maxTweets_ine = 100_000
date_initial = "2022-01-01"

# Get tweets
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#INENoSeToca').get_items()): # se puede añadir esto --> since:'+date_initial
        if i>maxTweets_ine-1:
            break
        tweets_list_ine.append([tweet.user.username, tweet.date, tweet.id, tweet.content, tweet.url, tweet.lang,
                    tweet.hashtags, tweet.likeCount, tweet.replyCount, tweet.retweetCount, tweet.quoteCount])
        # Otra info que puede sernos útil: tweet.media,  tweet.url

CPU times: user 12.8 s, sys: 1.48 s, total: 14.3 s
Wall time: 8min 36s


In [20]:
# Pandas dataframe con tweets que mencionen el hashtag #INENoSeToca
df_ine = pd.DataFrame(tweets_list_ine, columns=column_names)
df_ine['date'] = df_ine['date'].dt.strftime('%Y-%m-%d')
df_ine

,username,date,id,content,url,language,hashtags,likes_count,reply_count,retweet_count,quote_count
0,finkmann_me,2022-11-17,1593047502890369024,@MARIACLEMENTEMX @lopezobrador_ @PartidoMorena...,https://twitter.com/finkmann_me/status/1593047...,es,"[ElPeorGobiernoDeLaHistoria, ElPeorPresidenteD...",0,0,0,0
1,gafer26,2022-11-17,1593046320910340096,Eres tan INEpto @lopezobrador_ que no entiende...,https://twitter.com/gafer26/status/15930463209...,es,[INENoSeToca],0,0,0,0
2,roosterjau,2022-11-17,1593043428174139395,#INE #INENoSeToca https://t.co/On929dLtGN,https://twitter.com/roosterjau/status/15930434...,qht,"[INE, INENoSeToca]",0,0,0,0
3,Elytan0742Farie,2022-11-17,1593041777526984706,"@SandraCuevas_ @lopezobrador_ Jajaja, el presi...",https://twitter.com/Elytan0742Farie/status/159...,es,"[INENoSeToca, ladypelotas]",0,0,0,0
4,quehorastraen,2022-11-17,1593037500603576320,@micm54 La Marcha del 13 de noviembre no fue d...,https://twitter.com/quehorastraen/status/15930...,es,[INENoSeToca],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
16027,EnriqueBedolla,2019-11-10,1193411453878845441,#INENoSeToca \n\n#INEUltimoBastion https://t.c...,https://twitter.com/EnriqueBedolla/status/1193...,qht,"[INENoSeToca, INEUltimoBastion]",0,0,1,0
16028,superharta99,2019-11-10,1193388698269896705,#INENoSeToca https://t.co/DNRHXgwJhG,https://twitter.com/superharta99/status/119338...,qme,[INENoSeToca],1,0,2,0
16029,superharta99,2019-11-10,1193388629483229184,#INENoSeToca,https://twitter.com/superharta99/status/119338...,qht,[INENoSeToca],0,0,1,0
16030,EnriqueBedolla,2019-11-10,1193387134922543109,#INENoSeToca https://t.co/T8mjiEEFXO,https://twitter.com/EnriqueBedolla/status/1193...,qht,[INENoSeToca],1,0,4,0


In [23]:
# ejemplo de tweet
df_ine['content'][1]

'Eres tan INEpto @lopezobrador_ que no entiendes que la marcha del domingo fue en defensa del #INENoSeToca  no en tu contra. https://t.co/NRehxvXsi5'

In [26]:
source = pd.DataFrame(df_ine['date'].value_counts()).reset_index().rename(columns={'index':'fecha', 'date':"tweets"})
source.head()

,fecha,tweets
0,2022-11-13,6062
1,2022-11-14,2737
2,2022-11-09,771
3,2022-11-12,742
4,2022-11-11,737


In [31]:
alt.Chart(source[source['fecha'] >= "2022-01-01"]).mark_area(color="pink").encode(
    alt.X("fecha:T", title="2022"),
    alt.Y("tweets:Q", title="Número de Tweets", scale=alt.Scale(type="log")),
    tooltip=["fecha:T","tweets"]
).properties(
    title="Volumen de tweets con hashtag #INENoSeToca durante 2022"
)

alt.Chart(...)

# Referencias
- **API de Twitter**
    - [Solicitud de cuenta de desarrollador](https://developer.twitter.com/en/portal/dashboard)
    - [Twitter API Data Collection](https://www.youtube.com/watch?v=Jl-_dDqSaUQ&t=59s) por Stevesie Data (minuto 0:59 al 1:44)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v1_TwitterAPI.ipynb)  

<br>

- **librería `GetOldTweets`**
    - [Descripción del paquete](https://pypi.org/project/GetOldTweets3/) 
    - [Página del paquete en GitHub](https://github.com/Mottl/GetOldTweets3)
    - [Error del paquete](https://github.com/Mottl/GetOldTweets3/issues/98)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v2_GetOldTweets3.ipynb)  

<br>

- **librería `snscrape`**
    - [Descripción del paquete](https://pypi.org/project/snscrape/) 
    - [Página del paquete en GitHub](https://github.com/JustAnotherArchivist/snscrape)
    - [Ejemplo de uso](https://github.com/vcuspinera/Canada_response_covid/blob/master/src/twitter-search_v3_snscrape.ipynb)  

<br>

- **Artículos**
    - [How to scrape millions of tweets using snscrape](https://medium.com/dataseries/how-to-scrape-millions-of-tweets-using-snscrape-195ee3594721) por Rashi Desai.
    - [How to Scrape Tweets With snscrape](https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af) por Martin Beck.